In [ ]:
%%writefile streamlit_app1.py
import streamlit as st
import pandas as pd  # Pandas kütüphanesini içe aktarma
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from matplotlib.ticker import PercentFormatter



df = pd.read_csv("Top_Seller_Cleaned_temp_2.csv")  # Doğru CSV dosya yolunu girin

# Tüm uygulamanın arka plan rengini ayarlayan stil bilgisini ekleyin.
st.markdown(
    """
    <style>
    .stApp {
        background-color: orange;  /* Koyu sarı renk */
    }
    .custom-text {
        font-size: 24px; /* Yazı büyüklüğü */
        color: red;      /* Yazı rengi */
    }
    </style>
    """,
    unsafe_allow_html=True
)


# HTML ve CSS ile sağ üst köşeye logo eklemek için bir fonksiyon

def add_logo(image_path, height='50px'):
    st.markdown(
        f"""
        <div style="position: absolute; top: 0; right: 0; padding: 10px;">
            <img src="{image_path}" style="height: {height};">
        </div>
        """,
        unsafe_allow_html=True
    )


# Sayfa yapısını oluşturmak için bir sayfa yöneticisi yapısı kullanacağız.
def Sales_Rank_Analizi():
    add_logo("https://ao-pub-files.s3.eu-central-1.amazonaws.com/public/academy/epaBEy4Dm4kf8LbZhqx4q7feBfp4gz5weFkKSfyX.png",'50px') 
    st.title("Sales Rank Analizi")
    
     # Filtrelenmiş veri çerçevesini oluşturun.
    filtered_df = df[df['Categories: Root'].isin(['Tools & Home Improvement', 'Home & Kitchen'])]

    # Her mağaza için her ana kategoriye ait 'Sales Rank: 30 days avg.' ortalamalarını hesaplayın.
    store_category_rank_30_avg = filtered_df.groupby(['Store_Name', 'Categories: Root'])['Sales Rank: 30 days avg.'].mean().reset_index()
    
    # Özel renk ölçeği oluşturun.
    color_discrete_map = {'Tools & Home Improvement': 'red', 'Home & Kitchen': 'blue'}

    # category_orders parametresini kullanarak sütun sıralamasını belirleyin.
    category_orders = {"Categories: Root": ["Tools & Home Improvement", "Home & Kitchen"]}

    # Grafik oluşturmak için Plotly Express kullanın.
    fig = px.bar(store_category_rank_30_avg,
                 x='Store_Name',
                 y='Sales Rank: 30 days avg.',
                 color='Categories: Root',
                 title='Mağazalara ve Ana Kategorilere Göre 30 Günlük Ortalama Sales Rank',
                 labels={'Sales Rank: 30 days avg.': '30 Günlük Ortalama Sales Rank'},
                 hover_data=['Categories: Root'],
                 color_discrete_map=color_discrete_map,
                 category_orders=category_orders)  # Renk eşlemesini ve sıralamasını kullanın

    # Eksen isimlerini ve grafik tasarımını güncelleyin.
    fig.update_layout(xaxis_title='Mağaza Adı',
                      yaxis_title='30 Günlük Sales Rank Ortalaması',
                      barmode='group',
                      xaxis={'categoryorder':'total descending'})

    # Streamlit'te grafik gösterimi yapın.
    st.plotly_chart(fig)


    # 'Bought in past month' sütununa göre en yüksek 20 satışı filtreleyin.
    top_bought_last_month = df.nlargest(20, 'Bought in past month')

    # Seaborn ile bir barplot oluşturun.
    plt.figure(figsize=(12, 10))
    bar_plot = sns.barplot(
        data=top_bought_last_month,
        y='ASIN',  # Y ekseninde ASIN gösterecek.
        x='Bought in past month',  # X ekseninde bu ay satılan miktarı gösterecek.
        hue='Categories: Root',  # Kategori bilgileri renk olarak ayrılacak.
        dodge=False,  # Kategoriler üst üste gözükecek şekilde.
        palette='viridis'  # Renk paleti.
)

    # Grafiğin X ve Y eksenindeki etiketlerini ve başlığını ayarlayın.
    plt.xlabel('Bought in Past Month')
    plt.ylabel('ASIN')
    plt.title('Top 20 Products Bought in the Last Month by Subcategory and Sales Rank 30 Days Avg')

    # Her bir bar için 'Sales Rank: 30 days avg.' değerini barın ortasında beyaz renkle göster.
    for index, (value, rank) in enumerate(zip(top_bought_last_month['Bought in past month'], top_bought_last_month['Sales Rank: 30 days avg.'])):
        plt.text(value / 2, index, f'{rank:.2f}', color='white', ha='center', va='center')

    # Efsaneyi sağa alın.
    plt.legend(title='Rootcategory', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Layout'u düzeltmek için.
    plt.tight_layout()

    # Grafiği Streamlit'e gömmek için.
    st.pyplot(plt)
    
    st.write("""Bu grafikte son ay en çok satan 20 ASIN in sales rank 30 days avg. değerlerini ve bu asinlerin Root categorilerini burada görebiliyoruz. 17 ürünün "Home & Kitchen" root kategorisine ait olduğunu görüyoruz. Sales rank score larını bu iki kategoride karşılaştırırsak home&Kitchen a ait olan ürünlerin değerlerinin daha düşük olduğunu burada bu ürünlerin satış miktarlarının çok daha fazla olduğunu söyleyebiliriz""")
    
    # Öncelikle, sadece istediğimiz kategorilere ait satırları filtreleyelim
    filtered_df = df[df['Categories: Root'].isin(['Tools & Home Improvement', 'Home & Kitchen'])]

    # Şimdi, Store_Name ve Categories: Root sütunlarına göre gruplama yapıyoruz ve her bir grup için satış sayısını hesaplıyoruz.
    store_root_category_counts = filtered_df.groupby(['Store_Name', 'Categories: Root']).size().reset_index(name='Product_Count')

    # Sonuçları, Store_Name ve Categories: Root sütunlarına göre sıralayarak görüntüleyelim.
    store_root_category_counts_sorted = store_root_category_counts.sort_values(by=['Store_Name', 'Categories: Root'])

    # Veriyi 'Categories: Root' sütununa göre pivot ederek yığılmış çubuk grafiği için hazırlıyoruz
    pivot_data = store_root_category_counts_sorted.pivot(index='Store_Name', columns='Categories: Root', values='Product_Count').reset_index()

    # Seaborn kütüphanesinden "deep" renk paletini alıyoruz
    color_palette = sns.color_palette("deep", len(pivot_data.columns[1:])).as_hex()
    
    # Grafik oluşturma
    fig = go.Figure()
    
    for idx, category in enumerate(pivot_data.columns[1:]):
        fig.add_trace(go.Bar(
            x=pivot_data['Store_Name'],
            y=pivot_data[category],
            name=category,
            marker_color=color_palette[idx]
    ))
    
        fig.update_layout(
        title='Mağazaların Ana Kategoriye Göre Ürün Sayısı',
        xaxis_title='Mağaza Adı',
        yaxis_title='Ürün Sayısı',
        barmode='stack',
        template="plotly_white"
)


    # Grafik gösterimi.
    st.plotly_chart(fig)

    
    st.write("""**Yapılanlar:**

Burada ploty kütüphanesi kullanılarak birçok veri aynı anda görselleştirildi. Yukarıdaki grafikte mağazaları ve bu mağazaların ürünleri root kategorilerine ve buradaki ürün sayılarına göre çizdirildi.

**Bulgular:**

Layger, NorthernShipmens ve UnbetatableSale mağazalarının ürün sayıları ceşitliliğinin miktarı hemen gözümüze çarptı. Bunun yanında data setimizdeki bu mağazaların genel olarak Home en Kitchen en Tools & HomeImprovement root kategorilerindeki büyük bir ürün yelpazesi sunduğu görüldü.
 

**Sonraki Adımlar:**

Belirli kategorilerde ürün sayılarına göre pazar payı analizi yapılabilir.

Kategorilere göre satış verileri ve müşteri tercihleri incelenerek mağazaların pazarlama ve stok stratejileri geliştirilebilir.

**Tavsiyeler:**

Ürün çeşitliliği yüksek olan mağazalar, müşterilerine daha geniş bir seçenek sunarak rekabet avantajı sağlayabilir.

Az temsil edilen kategorilerde ürün sayısını artırarak bu niş pazarlarda fark yaratılabilir.""")

    
def Product_Review_Analizi():
    # Logo ekleme fonksiyonu
    add_logo("https://ao-pub-files.s3.eu-central-1.amazonaws.com/public/academy/epaBEy4Dm4kf8LbZhqx4q7feBfp4gz5weFkKSfyX.png", '40px') 
    st.title("Product Review Analizi")
     
    # Sidebar'da bir slider oluşturun ve değeri user_input_slider değişkenine atayın
    user_input_slider = st.sidebar.select_slider(
        'Review Trend Analizi için Ürün Sayısını Seçin',
        options=[20, 50, 100, 500, 1000, 'all'],
        value='all'  # Varsayılan değer
    )


    # İnceleme trendini gösteren fonksiyonun tanımı
    def show_review_trend_2(df_r, user_input):
        # "Reviews" ile ilgili sütunları seç
        review_columns = [col for col in df_r.columns if "Reviews" in col]
        # ASIN ve "Reviews" ile ilgili sütunları içeren yeni bir DataFrame oluştur
        df_reviews = df_r[['ASIN'] + review_columns].copy()
        # Eksik değerleri temizle
        df_reviews_cleaned = df_reviews.dropna(subset=review_columns)
        # ASIN değerleri aynı olan satırların "Review" ile ilgili sütunlardaki değerlerinin ortalamasını al
        df_reviews_grouped = df_reviews_cleaned.groupby('ASIN').mean().reset_index()
        
        # Review trend skorunu hesapla
        df_reviews_grouped['review_trend'] = (
            df_reviews_grouped['Reviews: Review Count - 30 days avg.'] - 
            df_reviews_grouped['Reviews: Review Count - 90 days avg.']
        )
        
        # Review trend skorunu 0-1 arasında ölçeklendir
        scaler = MinMaxScaler(feature_range=(0, 1))
        df_reviews_grouped['review_trend_scaled'] = scaler.fit_transform(df_reviews_grouped[['review_trend']])
        
        # Tüm ürünleri veya kullanıcı girdisine göre en yüksek skorlu ürünleri seç
        if user_input == 'all':
            top_products = df_reviews_grouped
            plot_count = len(df_reviews_grouped)
        else:
            top_products = df_reviews_grouped.nlargest(user_input, 'review_trend_scaled')
            plot_count = user_input
        
        top_products = top_products.sort_values(by='review_trend_scaled', ascending=False)
        
        # Scatter plot çizdir, sadece en yüksek skorlu ürünleri göster
        fig = px.scatter(top_products.head(plot_count), x='ASIN', y='review_trend_scaled',
                         title=f'Top {plot_count} Products (Review Trend Score-2)',
                         labels={'review_trend_scaled': 'Review Trend Skoru (Ölçeklendirilmiş)',
                                 'ASIN': 'Ürün ID'},
                         height=500, symbol_sequence=['star'])
        fig.update_yaxes(range=[0, 1])
        
        return top_products, fig

    # Fonksiyonu kullanarak dönüş değerlerini değişkenlere ata
    df_r, review_trend_fig = show_review_trend_2(df, user_input=user_input_slider)
    
    # Streamlit'te veri çerçevesini ve grafiği göster
    st.write("En İyi Ürünler (Review Trend Skoru-2)")
    st.plotly_chart(review_trend_fig)
    st.dataframe(df_r.head()) # İlk 5 satırı göster
    

    st.write(""""Grafik, yaklaşık 28.900 ürünü Plotly'de 'Review Trend Score-2'ye göre büyükten küçüğe olacak şekilde bir scatter plot olarak dağılımını göstermektedir. Yatay eksende ASIN'ler, dikey eksende ise 'Trend Review Score' bulunmaktadır. Ölçeklendirme olarak 'MinMaxScaler' kullanıldığı için dikey eksen 0-1 arasında ölçeklenmiştir. Değerlerin ortalaması yaklaşık 0.7 olarak tespit edilmiştir. Baştaki ve sondaki değerler aykırı değer olarak kabul edilebilir. Ancak buradaki amaç, en yüksek değere sahip ilk 100, 500 veya 1000 ürün arasından 'Review Rating'i de yüksek olan ürünleri seçmek ve satışa sunmak olabilir.""")

    

def Fiyat_Analizi():
    add_logo("https://ao-pub-files.s3.eu-central-1.amazonaws.com/public/academy/epaBEy4Dm4kf8LbZhqx4q7feBfp4gz5weFkKSfyX.png",'60px') 
    st.title("Fiyat Analizi")
 
   # Fiyat aralıklarını ve etiketleri belirle
    price_bins = [0, 50, 100, 300, float('inf')]
    price_labels = ['0-50', '50-100', '100-300', '300+']
    
    # Fiyat sütunlarını ve başlıkları belirle
    price_columns = [
    ('New: Current', 'New Current: Fiyat Aralıklarına Göre Ürün Sayısı'),
    ('Buy Box: Current', 'Buy Box: Fiyat Aralıklarına Göre Ürün Sayısı'),    
    ('New, 3rd Party FBM: Current', 'FBM: Fiyat Aralıklarına Göre Ürün Sayısı'),
    ('Amazon: Current', 'Amazon: Fiyat Aralıklarına Göre Ürün Sayısı')
]


     # Renkleri belirle
    colors = ['blue', 'green', 'red', 'purple']

    northern_shipments_data = df[df['Store_Name'] == 'NorthernShipments']
     # Grafik oluştur
    fig = go.Figure()
    
    for (column, title), color in zip(price_columns, colors):
        fbm_prices = northern_shipments_data[column]
        price_groups = pd.cut(fbm_prices, bins=price_bins, labels=price_labels)
        price_counts = price_groups.value_counts()
        fig.add_trace(go.Bar(x=price_counts.index, y=price_counts, name=title, marker=dict(color=color), text=price_counts, textposition='auto'))

        fig.update_layout(barmode='group', height=800, width=1000, title_text="NorthernShipments: Farklı Fiyat Aralıklarına Göre Ürün Sayısı")
        fig.update_xaxes(title_text="Fiyat Aralığı")
        fig.update_yaxes(title_text="Ürün Sayısı")


      # Grafik göster
    st.plotly_chart(fig)
    st.write("Northern Shipment: Genel patternin aksine 0-50 aralığındaki ürünler bu mağazamızda ikinci sırada yer almaktadır. En çok ürünün gruplandığı aralık ise 50-100 dolar olduğu görülmüştür")
  
    # Fiyat aralıklarını ve etiketleri belirle
    price_bins = [0, 50, 100, 300, float('inf')]
    price_labels = ['0-50', '50-100', '100-300', '300+']

    # Fiyat sütunlarını ve başlıkları belirle
    price_columns = [
       ('New: Current', 'New Current: Fiyat Aralıklarına Göre Ürün Sayısı'),
       ('Buy Box: Current', 'Buy Box: Fiyat Aralıklarına Göre Ürün Sayısı'),    
       ('New, 3rd Party FBM: Current', 'FBM: Fiyat Aralıklarına Göre Ürün Sayısı'),
       ('Amazon: Current', 'Amazon: Fiyat Aralıklarına Göre Ürün Sayısı')
]

    #Renkleri belirle
    colors = ['blue', 'green', 'red', 'purple']
    
    Layger_data = df[df['Store_Name'] == 'Layger']
    # Grafik oluştur
    fig = go.Figure()

    for (column, title), color in zip(price_columns, colors):
        fbm_prices = Layger_data[column]
        price_groups = pd.cut(fbm_prices, bins=price_bins, labels=price_labels)
        price_counts = price_groups.value_counts()
        fig.add_trace(go.Bar(x=price_counts.index, y=price_counts, name=title, marker=dict(color=color), text=price_counts, textposition='auto'))

    fig.update_layout(barmode='group', height=800, width=1000, title_text="Layger: Farklı Fiyat Aralıklarına Göre Ürün Sayısı")
    fig.update_xaxes(title_text="Fiyat Aralığı")
    fig.update_yaxes(title_text="Ürün Sayısı")

    # Grafik göster
    st.plotly_chart(fig)
    st.write("Layger: Mağaza bazında incelendiğinde en çok ürün çeşitliliği bulunan layger mağazasında da 0-50 dolar aralığındaki ürünler çoğunluktadır.")
    # Fiyat aralıklarını ve etiketleri belirle
    price_bins = [0, 50, 100, 300, float('inf')]
    price_labels = ['0-50', '50-100', '100-300', '300+']

    # Fiyat sütunlarını ve başlıkları belirle
    price_columns = [
       ('New: Current', 'New Current: Fiyat Aralıklarına Göre Ürün Sayısı'),
       ('Buy Box: Current', 'Buy Box: Fiyat Aralıklarına Göre Ürün Sayısı'),    
       ('New, 3rd Party FBM: Current', 'FBM: Fiyat Aralıklarına Göre Ürün Sayısı'),
       ('Amazon: Current', 'Amazon: Fiyat Aralıklarına Göre Ürün Sayısı')
]

    # Renkleri belirle
    colors = ['blue', 'green', 'red', 'purple']

    # Grafik oluştur
    fig = go.Figure()

    for (column, title), color in zip(price_columns, colors):
        fbm_prices = df[column]
        price_groups = pd.cut(fbm_prices, bins=price_bins, labels=price_labels)
        price_counts = price_groups.value_counts()
        fig.add_trace(go.Bar(x=price_counts.index, y=price_counts, name=title, marker=dict(color=color), text=price_counts, textposition='auto'))

    fig.update_layout(barmode='group', height=800, width=1000, title_text="Farklı Fiyat Aralıklarına Göre Ürün Sayısı")
    fig.update_xaxes(title_text="Fiyat Aralığı")
    fig.update_yaxes(title_text="Ürün Sayısı")

    # Grafik göster
    st.plotly_chart(fig)

    st.write("Genel Analiz: Top seller datasının bütününe bakıldığında satıcılar tarafından en çok 0-50 dolar aralığındaki ürünlerde başarılı oldukları görülmüştür")

def Buy_Box_Fiyat_Analizi():
    add_logo("https://ao-pub-files.s3.eu-central-1.amazonaws.com/public/academy/epaBEy4Dm4kf8LbZhqx4q7feBfp4gz5weFkKSfyX.png",'40px') 
    st.title("Buy Box Fiyat Analizi")
    
    # Veri filtreleme
    matched_seller_df = df[df['Store_Name'] == df['Buy Box Seller_Seller_Name']]
    matched_fba_df = df[df['Store_Name'] == df['Lowest FBA Seller_Seller_Name']]
    matched_fbm_df = df[df['Store_Name'] == df['Lowest FBM Seller_Seller_Name']]
    
    # Üçlü bar plot çizimi
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x='Comparison Type', y='Positive Feedback %', data=pd.concat([
         pd.DataFrame({'Comparison Type': 'Buy Box Seller', 'Positive Feedback %': matched_seller_df['Buy Box Seller_Positive_Feedback']}),
         pd.DataFrame({'Comparison Type': 'FBA Lowest Seller', 'Positive Feedback %': matched_fba_df['Lowest FBA Seller_Positive_Feedback']}),
         pd.DataFrame({'Comparison Type': 'FBM Lowest Seller', 'Positive Feedback %': matched_fbm_df['Lowest FBM Seller_Positive_Feedback']})
]), ci=None)

    plt.xlabel('Comparison Type')
    plt.ylabel('Positive Feedback %')
    plt.title('Positive Feedback Comparison for Matching Sellers')

    # Çubukların üzerindeki yüzde değerlerini büyüterek yazdır
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.1f}%', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                fontsize=17,  # Yazı büyüklüğü
                color='black', 
                xytext=(0, 6),  # Yazının konumu
                textcoords='offset points')


    # x eksenindeki yazıları büyüt
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=17)

    st.pyplot(plt)
    st.write("""
Analiz, mağaza bazında ele alınarak, aynı mağaza içindeki ürünlerin kaçında “buy_box” sahibi olduğunu ortaya koymaktadır. Buy box sahibi olmayan ürünler için ise, öncelikle FBA seller ve FBM seller arasındaki kıyaslamalar yapılmıştır. Bu karşılaştırmalar, positive seller feedback’leri üzerinden incelenmiştir.
Sonuçlar, en yüksek seller positive feedback’in %91.2 oranıyla FBA lowest seller’a ait olduğunu göstermektedir. Ancak, top seller’ların genel olarak yüksek pozitif feedback oranına sahip olduğu gözlemlenmiştir. Bu analiz, bir satıcının aynı anda buy box seller, FBA lowest seller ve FBM lowest seller olabileceği gerçeğini dikkate alarak yapılmıştır. Ancak, yüksek satıcı puanlarının buy box sahibi olma olasılığı üzerinde olumlu bir etkisi olduğu gözlemlenmiştir. Bu durum, satıcı puanlarının genel itibariyle bir satıcının performansını ve güvenilirliğini yansıttığını göstermektedir. Yüksek satıcı puanları, müşteri memnuniyetini ve olumlu geri bildirimleri temsil edebilir, bu da buy box sahibi olma şansını artırabilir.
Ancak, bu analizde belirtilen diğer faktörlerin (FBA lowest seller, FBM lowest seller, vb.) de buy box sahibi olma üzerinde etkisi olduğu unutulmamalıdır. Bu nedenle, bir satıcının buy box sahibi olma başarısını değerlendirmek için birden çok faktörün dikkate alınması önemlidir.
Bu durum, feedback ve satış performansının farklı satış stratejilerine dayalı olarak değişebileceğini göstermektedir.""")
def İlgili_Diğer_Featureların_Analizi():
    add_logo("https://ao-pub-files.s3.eu-central-1.amazonaws.com/public/academy/epaBEy4Dm4kf8LbZhqx4q7feBfp4gz5weFkKSfyX.png",'30px') 
    st.title("Top %1'e Giren Ürünlerin Analizi")
            # HTML ve CSS kullanarak yazı boyutunu biraz daha küçültme
    st.markdown("""
    <style>
    .moderate-font {
        font-size:18px;  # Yazı boyutunu 22px olarak ayarla
    }
    </style>
    """, unsafe_allow_html=True)

    st.markdown("""
    <div class="moderate-font">Kendi alt kategorisinde Top %1’lik dilime giren ürünler, Home & Kitchen ve Tools & Home Improvement kategorileri için Northern Shipments ve Layger firmaları özelinde incelenmiş ve tablodaki sonuçlara ulaşılmıştır:</div>
    """, unsafe_allow_html=True)

    
    
    st.image("image2.png", use_column_width=True)
       # HTML ve CSS kullanarak yazı boyutunu biraz daha küçültme
    st.markdown("""
    <style>
    .moderate-font {
        font-size:18px;  # Yazı boyutunu 22px olarak ayarla
    }
    </style>
    """, unsafe_allow_html=True)

    st.markdown("""
    <div class="moderate-font">Northern Shipments” firmasının "Home & Kitchen" grubunda  %1’lik dilimde sadece 6 tane ürünü varken, "Layger" firmasının 103 alt kategoride toplam 303 adet ürünü bulunmaktadir. 
Benzer şekilde “Tools & Home Improvement” kategorisinde %1’lik dilimde “Northern Shipments” firmasının hiç ürünü yokken, "Layger" fırmasının "Home & Kitchen" grubunda 98 alt kategoride toplam 214 adet ürünü bulunmaktadir. 
İşte bu bir başarıdır. Cünkü çok ürünü olup, altlarda da yer alabilirdi ama hem çok ürünü olup, hemde bunların önemli bir kısmını Top %1'de tutuyor olmaları, oneAMZ satıcıları için de örnek alınmalıdır.r:</div>
    """, unsafe_allow_html=True)
    
def Featureların_Birbirleri_İle_Etkileşiminin_Analizi():
    add_logo("https://ao-pub-files.s3.eu-central-1.amazonaws.com/public/academy/epaBEy4Dm4kf8LbZhqx4q7feBfp4gz5weFkKSfyX.png",'40px') 
    st.title("Featureların Birbirleri İle Etkileşiminin Analizi")
     # "Home & Kitchen" ve "Tools & Home Improvement" kategorilerini filtrele
    home_kitchen_df = df[df['Categories: Root'] == 'Home & Kitchen']
    tools_home_df = df[df['Categories: Root'] == 'Tools & Home Improvement']
    
    # Her kategori için 'Store_Name' sütununa göre grupla ve istatistikleri hesapla
    hk_stats = home_kitchen_df.groupby('Store_Name').agg({
        'Reviews: Review Count': 'mean', 
        'Reviews: Rating': 'mean', 
                'ASIN': 'count'
    }).rename(columns={'ASIN': 'Total Products'}).sort_values(by='Reviews: Review Count', ascending=False).head(20)

    thi_stats = tools_home_df.groupby('Store_Name').agg({
        'Reviews: Review Count': 'mean', 
        'Reviews: Rating': 'mean', 
        'ASIN': 'count'
    }).rename(columns={'ASIN': 'Total Products'}).sort_values(by='Reviews: Review Count', ascending=True).head(20)

    # Subplot oluştur (dikey piramit)
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.02, subplot_titles=("Home & Kitchen", "Tools & Home Improvement"))

    # "Home & Kitchen" yatay barlar (ortalama inceleme sayısı) - Yellow
    fig.add_trace(
        go.Bar(
            y=hk_stats.index,
            x=-hk_stats['Reviews: Review Count'],
            orientation='h',
            name='Average Review Count',
            text=hk_stats['Reviews: Review Count'].round(1).astype(str),
            marker=dict(color='yellow', line=dict(color='black', width=1)),
            offsetgroup=0,
        ),
        row=1, col=1
    )
    
    #"Tools & Home Improvement" yatay barlar (ortalama inceleme sayısı) - Yellow

    fig.add_trace(
        go.Bar(
            y=thi_stats.index,
            x=thi_stats['Reviews: Review Count'],
            orientation='h',
            name='Average Review Count',
            text=thi_stats['Reviews: Review Count'].round(1).astype(str),
            marker=dict(color='yellow', line=dict(color='black', width=1)),
            offsetgroup=2,
        ),
        row=1, col=2
    )

    # "Home & Kitchen" yatay barlar (toplam ürün sayısı) - Green
    fig.add_trace(
        go.Bar(
            y=hk_stats.index,
            x=-hk_stats['Total Products'],
            orientation='h',
            name='Total Products',
            text=hk_stats['Total Products'].astype(str),
            marker=dict(color='green', line=dict(color='black', width=1)),
            offsetgroup=1,
        ),
        row=1, col=1
    )

    # "Tools & Home Improvement" yatay barlar (toplam ürün sayısı) - Green
    fig.add_trace(
        go.Bar(
            y=thi_stats.index,
            x=thi_stats['Total Products'],
            orientation='h',
            name='Total Products',
            text=thi_stats['Total Products'].astype(str),
            marker=dict(color='green', line=dict(color='black', width=1)),
            offsetgroup=3,
        ),
        row=1, col=2
    )
    
    # Layout ayarları
    fig.update_layout(
        title_text='Average Review Count and Total Products - Tornado Plot',
        width=1000,
        height=900,
        barmode='group',
        yaxis=dict(title='Store Name', autorange='reversed'),
        xaxis=dict(title='Average Review Count', showticklabels=True),
        xaxis2=dict(title='Average Review Count'),
        showlegend=True,
        legend=dict(
           x=0.1, 
           y=0.1, 
           orientation='v',  # Yatay konumlandırma
           traceorder='normal'  # İz sırasına göre düzenle
    )
)

    # Streamlit'te grafiği göster
    st.plotly_chart(fig)

    st.write(""""Plotly kütüphanesinde 'Tornado' türünde hazır bir grafik olmamasına rağmen, biz bunu ChatGPT 4.0'ın yardımıyla hazırladık. Grafiğin yatay ekseninde sağa ve sola doğru 'Average Review Count' değerleri, dikey eksende ise en çok satan rakip satıcılar görülmektedir. Grafiğin sol tarafı 'Home & Kitchen', sağ tarafı 'Tools & Home Improvement' ana kategorisine aittir. Sarı barlar 'Average Review Count'u, yeşil barlar ise mağazaların 'Total Products', yani ürün sayısını göstermektedir. Grafik, mağazaları 'Home & Kitchen' ana kategorisindeki 'Average Review Count'larına göre yukarıdan aşağıya sıralamaktadır. Yani başarı ölçütü olarak ürün başına düşen ortalama inceleme sayısı alınmıştır.""")
    

# Sidebar başlığını HTML ve CSS ile renkli olarak ekleyin
st.sidebar.markdown(
    """
    <h1 style='color: orange;'>DATA ANALIST SUNUMU INT4</h1>
    """, 
    unsafe_allow_html=True
)

# Sidebar title with increased font size and bold font
st.sidebar.markdown(
    """
    <style>
    .big-font {
        font-size: 22px;
        font-weight: bold;
    }
    </style>
    <div class='big-font'>Top Seller Keepa</div>
    """, 
    unsafe_allow_html=True
)

# Correct sidebar radio options with proper comma separation
page = st.sidebar.radio(
    "",
    ['Sales Rank Analizi', 'Product Review Analizi', 'Fiyat Analizi', 'Buy Box Fiyat Analizi', "Top %1'e Giren Ürünlerin Analizi", "Feature'ların Birbirleri İle Etkileşiminin Analizi"]
)

# Main part
if __name__ == "__main__":
    if page == 'Sales Rank Analizi':
        Sales_Rank_Analizi()
    elif page == 'Product Review Analizi':
        Product_Review_Analizi()
    elif page == 'Fiyat Analizi':
        Fiyat_Analizi()
    elif page == 'Buy Box Fiyat Analizi':
        Buy_Box_Fiyat_Analizi()
    elif page == "Top %1'e Giren Ürünlerin Analizi":
        İlgili_Diğer_Featureların_Analizi()
    elif page == "Feature'ların Birbirleri İle Etkileşiminin Analizi":
        Featureların_Birbirleri_İle_Etkileşiminin_Analizi()

In [ ]:
!streamlit run streamlit_app1.py